## Prediction model w/ PCA
The data were transformed with principal components analysis (PCA). The feature correlation matrix and component's variance were shown together.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
#for performance plots
import matplotlib.pyplot as plt

In [ ]:
train_set = np.array(pd.read_csv('data/train_set.csv'))
test_set = np.array(pd.read_csv('data/test_set.csv'))

nFeat = len(train_set[0,:]) - 2

X_train = train_set[:,:nFeat]
Y_train = train_set[:,nFeat:nFeat+2]
X_test = test_set[:,:nFeat]
Y_test = test_set[:, nFeat:nFeat+2]

In [ ]:
## correlation matrix for checking relative importance of features
feature_names = ['A (carbon)', 'B (carbon)', 'A(L)','A(B1)', 'A(B5)', 'B(L)','B(B1)', 'B(B5)', 'eps', 'temp']

ax = plt.axes()
pca = PCA(n_components=4)
components = pca.fit(X_train).components_.T
vmax = np.abs(components).max()
im=ax.imshow(components, cmap="RdBu_r", vmax=vmax, vmin=-vmax)

ax.set_yticks(np.arange(len(feature_names)))
ax.set_yticklabels(feature_names)
ax.set_xticks([0,1,2,3])
ax.set_xticklabels(["PC1", "PC2","PC3","PC4"])

plt.colorbar(im).ax.set_ylabel("$r$", rotation=0)
plt.tight_layout()

In [ ]:
## reducing data by using PCA

n_component=4
pca = PCA(n_components=n_component)
X_train = pca.fit_transform(X_train)
X_train = pd.DataFrame(data=X_train, columns = range(n_component))

In [ ]:
## confirmation of component's variance
prop_var = pca.explained_variance_ratio_
eigenvalues = pca.explained_variance_

PC_numbers = np.arange(pca.n_components_) + 1

print(prop_var)

plt.plot(PC_numbers, 
         prop_var, 
         'ro-')
plt.title('Figure 1: Scree Plot', fontsize=8)
plt.ylabel('Proportion of Variance', fontsize=8)
plt.show()

In [ ]:
X_test = pca.transform(X_test)
X_test = pd.DataFrame(data=X_test, columns = range(n_component))
X_train = X_train.values[:,:]
X_test = X_test.values[:,:]

In [ ]:
# scale the train and test set
scalerx = StandardScaler().fit (X_train)
X_train = scalerx.transform(X_train)
X_test  = scalerx.transform(X_test)

In [ ]:
## developing feed-forward neural network
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim
import torch.nn.functional as F

## construction of hidden layers
class FNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_component,256)
        self.fc2 = nn.Linear(256, 512)
        self.dp1 = nn.Dropout(p=0.4)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 256)
        self.fc5 = nn.Linear(256, 2)
        
    def forward(self, x):
        h = F.selu(self.fc1(x))
        h = F.selu(self.fc2(h))
        h = self.dp1(h)
        h = F.selu(self.fc3(h))
        h = F.selu(self.fc4(h))
        out = self.fc5(h)
        return out
    ## FNN model training    
    def fit(self, data_loader, criterion, optimizer):
        self.train()
        sum_train_losses = 0

        for data, target in data_loader:
            
            optimizer.zero_grad()
            pred = self(data)
            loss = criterion(pred, target)
            loss.backward()
            optimizer.step()

            sum_train_losses += loss.item()

        return sum_train_losses / len(data_loader)
    ## FNN model evaluation
    def predict(self, data_loader):
        self.eval()
        list_preds = list()

        with torch.no_grad():
            for data, _ in data_loader:
                pred = self(data)
                list_preds.append(pred)

        return torch.cat(list_preds, dim=0).cpu().numpy()

## define loss function    
def RMSELoss(pred,target):
    return torch.sqrt(torch.mean((pred-target)**2))
    

In [ ]:
## developing ML models
models = {}
models['KR rbf'] =GridSearchCV(KernelRidge(kernel='rbf'), cv=5, 
                               param_grid={'alpha': [10, 1, 0.1, 1e-2, 1e-3], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
                              scoring ='r2')
models['SVM rbf'] = GridSearchCV(SVR(kernel='rbf'), cv=5, 
                                 param_grid={'C': [0.001,0.01, 0.1, 1, 10,100], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1,10, 100], 'epsilon': [0.01, 0.1, 0.5, 1, 2, 4]}, 
                                 scoring ='r2')
models['RF'] = GridSearchCV(RandomForestRegressor(random_state=400),cv=5, 
                                 param_grid ={'max_depth': [5,10,15], 'n_estimators': [20,40,80,120]}, 
                                 scoring='r2')
models['XGBoost'] = GridSearchCV(xgb.XGBRegressor(), cv=5,
                                  param_grid ={'max_depth': [2,5,10,15], 'n_estimators': [20,40,80,120]},
                                  scoring='r2')
models['FNN'] = FNN()


for idx, model in enumerate(models):
    print('model: ', model)
    ## for FNN model, input data converts into torch datatype
    if model == 'FNN':
        X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.22, shuffle=True, random_state=42)
        
        X_train_t = torch.Tensor(X_train)
        X_val_t = torch.Tensor(X_val)
        X_test_t = torch.Tensor(X_test)
        Y_train_t = torch.Tensor(Y_train)
        Y_val_t = torch.Tensor(Y_val)
        Y_test_t = torch.Tensor(Y_test)

        data_train = TensorDataset(X_train_t, Y_train_t)
        data_val = TensorDataset(X_val_t, Y_val_t)
        data_test = TensorDataset(X_test_t, Y_test_t)
        train_loader = DataLoader(data_train, batch_size=64, shuffle=False)
        val_loader = DataLoader(data_val, batch_size=64, shuffle=False)
        test_loader = DataLoader(data_test, batch_size=64, shuffle=False)

        criterion = RMSELoss
        optimizer = optim.Adam(models[model].parameters())

        for epoch in range(500):
            models[model].train()
            train_loss = models[model].fit(train_loader, criterion, optimizer)
            models[model].eval()
            val_loss = models[model].fit(val_loader, criterion, optimizer)
           
        Y_pred = models[model].predict(test_loader)
        Y_test_t = Y_test_t.detach().numpy()
        
        r2_yield1 = r2_score(Y_test_t[:,0], Y_pred[:,0])
        r2_yield2 = r2_score(Y_test_t[:,1], Y_pred[:,1])
        mae1 = np.mean(np.abs(Y_pred[:,0]-Y_test_t[:,0]))
        mae2 = np.mean(np.abs(Y_pred[:,1]-Y_test_t[:,1]))
        print('R2 score:     {:.2}\t\t{:.2}'.format(r2_yield1, r2_yield2))
        print('MAE:\t     {:.2}\t\t{:.2}'.format(mae1, mae2))
    
    ## for other models
    else:
        models[model].fit(X_train, Y_train[:,0])
        Y_pred1 = models[model].predict (X_test)
        r2_yield1 = np.around(r2_score(Y_test[:,0], Y_pred1), 2)
        mae1 = np.around(np.mean(np.abs(Y_pred1 - Y_test[:,0])),1)
        models[model].fit(X_train, Y_train[:,1])
        Y_pred2 = models[model].predict (X_test)
        r2_yield2 = np.around(r2_score(Y_test[:,1], Y_pred2), 2)
        mae2 = np.around(np.mean(np.abs(Y_pred2 - Y_test[:,1])),1)
        print('R2 score:     {}\t\t{}'.format(r2_yield1, r2_yield2))
        print('MAE:\t     {}\t\t{}'.format(mae1, mae2))


## Prediction model w/o PCA
These models were evaluated with the data in the absence of the Sterimol values. Thus, 4 features were employed in the dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
#for performance plots
import matplotlib.pyplot as plt

In [ ]:
## omitting the Sterimol values
train_set = np.array(pd.read_csv('data/train_set.csv'))
train_set = np.delete(train_set, [
                                 2,3,4,5,6,7
                                 ], 1)
test_set = np.array(pd.read_csv('data/test_set.csv'))
test_set = np.delete(test_set, [
                               2,3,4,5,6,7
                               ], 1)
nFeat = len(train_set[0,:]) - 2

X_train = train_set[:,:nFeat]
Y_train = train_set[:,nFeat:nFeat+2]
X_test = test_set[:,:nFeat]
Y_test = test_set[:, nFeat:nFeat+2]

In [ ]:
# scale the train and test set
scalerx = StandardScaler().fit (X_train)
X_train = scalerx.transform(X_train)
X_test  = scalerx.transform(X_test)

In [ ]:
## developing feed-forward neural network
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim
import torch.nn.functional as F

## construction of hidden layers
class FNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(nFeat,256)
        self.fc2 = nn.Linear(256, 512)
        self.dp1 = nn.Dropout(p=0.4)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 256)
        self.fc5 = nn.Linear(256, 2)
        
    def forward(self, x):
        h = F.selu(self.fc1(x))
        h = F.selu(self.fc2(h))
        h = self.dp1(h)
        h = F.selu(self.fc3(h))
        h = F.selu(self.fc4(h))
        out = self.fc5(h)
        return out
    ## FNN model training    
    def fit(self, data_loader, criterion, optimizer):
        self.train()
        sum_train_losses = 0

        for data, target in data_loader:
            
            optimizer.zero_grad()
            pred = self(data)
            loss = criterion(pred, target)
            loss.backward()
            optimizer.step()

            sum_train_losses += loss.item()

        return sum_train_losses / len(data_loader)
    ## FNN model evaluation
    def predict(self, data_loader):
        self.eval()
        list_preds = list()

        with torch.no_grad():
            for data, _ in data_loader:
                pred = self(data)
                list_preds.append(pred)

        return torch.cat(list_preds, dim=0).cpu().numpy()

## define loss function    
def RMSELoss(pred,target):
    return torch.sqrt(torch.mean((pred-target)**2))
    

In [ ]:
## developing ML models
models = {}
models['KR rbf'] =GridSearchCV(KernelRidge(kernel='rbf'), cv=5, 
                               param_grid={'alpha': [10, 1, 0.1, 1e-2, 1e-3], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
                              scoring ='r2')
models['SVM rbf'] = GridSearchCV(SVR(kernel='rbf'), cv=5, 
                                 param_grid={'C': [0.001,0.01, 0.1, 1, 10,100], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1,10, 100], 'epsilon': [0.01, 0.1, 0.5, 1, 2, 4]}, 
                                 scoring ='r2')
models['RF'] = GridSearchCV(RandomForestRegressor(random_state=400),cv=5, 
                                 param_grid ={'max_depth': [5,10,15], 'n_estimators': [20,40,80,120]}, 
                                 scoring='r2')
models['XGBoost'] = GridSearchCV(xgb.XGBRegressor(), cv=5,
                                  param_grid ={'max_depth': [2,5,10,15], 'n_estimators': [20,40,80,120]},
                                  scoring='r2')
models['FNN'] = FNN()

for idx, model in enumerate(models):
    print('model: ', model)
    
    ## for FNN model, input data converts into torch datatype
    if model == 'FNN':
        X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.22, shuffle=True, random_state=42)
        
        X_train_t = torch.Tensor(X_train)
        X_val_t = torch.Tensor(X_val)
        X_test_t = torch.Tensor(X_test)
        Y_train_t = torch.Tensor(Y_train)
        Y_val_t = torch.Tensor(Y_val)
        Y_test_t = torch.Tensor(Y_test)

        data_train = TensorDataset(X_train_t, Y_train_t)
        data_val = TensorDataset(X_val_t, Y_val_t)
        data_test = TensorDataset(X_test_t, Y_test_t)
        train_loader = DataLoader(data_train, batch_size=64, shuffle=False)
        val_loader = DataLoader(data_val, batch_size=64, shuffle=False)
        test_loader = DataLoader(data_test, batch_size=64, shuffle=False)

        criterion = RMSELoss
        optimizer = optim.Adam(models[model].parameters())

        for epoch in range(500):
            models[model].train()
            train_loss = models[model].fit(train_loader, criterion, optimizer)
            models[model].eval()
            val_loss = models[model].fit(val_loader, criterion, optimizer)
           
        Y_pred = models[model].predict(test_loader)
        Y_test_t = Y_test_t.detach().numpy()
        
        r2_yield1 = np.around(r2_score(Y_test_t[:,0], Y_pred[:,0]), 2)
        r2_yield2 = np.around(r2_score(Y_test_t[:,1], Y_pred[:,1]), 2)
        mae1 = np.mean(np.abs(Y_pred[:,0]-Y_test_t[:,0]))
        mae2 = np.mean(np.abs(Y_pred[:,1]-Y_test_t[:,1]))
        print('R2 score:     {}\t\t{}'.format(r2_yield1, r2_yield2))
        print('MAE:\t     {:.2}\t\t{:.2}'.format(mae1, mae2))
        
    ## for other models
    else:

        models[model].fit(X_train, Y_train[:,0])
        Y_pred1 = models[model].predict (X_test)
        r2_yield1 = np.around(r2_score(Y_test[:,0], Y_pred1), 2)
        mae1 = np.around(np.mean(np.abs(Y_pred1 - Y_test[:,0])),1)
        
        models[model].fit(X_train, Y_train[:,1])
        Y_pred2 = models[model].predict (X_test)
        r2_yield2 = np.around(r2_score(Y_test[:,1], Y_pred2), 2)
        mae2 = np.around(np.mean(np.abs(Y_pred2 - Y_test[:,1])),1)
        print('R2 score:     {}\t\t{}'.format(r2_yield1, r2_yield2))
        print('MAE:\t     {}\t\t{}'.format(mae1, mae2))
